### Assignment 2 Understanding transfer learning and fine tuning 

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data import Dataset
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
from torchvision import datasets
from torch.utils.data import random_split
from torchvision import datasets
import os 
import numpy as np 
from typing import Literal
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import accuracy_score

print(f"[INFO] Torch infos: {torch.__version__}")


[INFO] Torch infos: 2.10.0+cpu


# We need transform from resnet18 and its weights


In [4]:
transform = ResNet18_Weights.DEFAULT.transforms()

# pick a dataset  you can import from

```python
from torchvision import datasets
```

# Easy picks:

- Food101
- Flowers102
- DTD
- FGVAircraft

# Other picks:

```python

full_train = datasets.OxfordIIITPet(root="data", split="trainval", download=True, transform=transform)
test_ds    = datasets.OxfordIIITPet(root="data", split="test", download=True, transform=transform)

```


In [ ]:
full_train = datasets.FGVCAircraft('../Dataset', split="trainval", transform=transform, download=True)
test_ds = datasets.FGVCAircraft('../Dataset', split="test", download=True, transform=transform)



6667


# Split the data

In [9]:
train_ratio = 0.8
train_size = int(len(full_train) * train_ratio)

validation_size = len(full_train) - train_size

train_ds, validation_ds = random_split(full_train, [train_size, validation_size])

print("Train Size:", train_size)
print("Validation Size:", validation_size)
print("Train and Validation Ds", train_ds, validation_ds)

Train Size: 5333
Validation Size: 1334
Train and Validation Ds <torch.utils.data.dataset.Subset object at 0x000001CB66A32610> <torch.utils.data.dataset.Subset object at 0x000001CB6634EB90>


# load the train, validation and test split 

In [ ]:

print(f"Training set size: {len(train_ds)} samples")
print(f"Validation set size: {len(validation_ds)} samples")
print(f"Test set size: {len(test_ds)} samples")

assert len(train_ds) + len(validation_ds) == len(full_train)


Training set size: 5333 samples
Validation set size: 1334 samples
Test set size: 3333 samples


# create dataloader handler from the dataset

In [18]:
train_loader = DataLoader(train_ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)
validation_loader = DataLoader(validation_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_ds, batch_size=256, shuffle=False, num_workers=2, pin_memory=True)

# now create the cnn, use the resnet18 as backbone, and ResNet18_Weights as initial weights.
- create the backbone
- create a classifier 
- based on your dataset add the correct number of output classes
- the classifier have to be trainable.

# Question 1?
- Why we freeze the backbone? Why not the classifier?

# add the train evaluation, and predict functions here

### Phase 1: Transfer learning

# freeze all layers except the classifier.
- train and evaluate the model for 50 epochs 
- remember to save val loss and train loss

# Plot accuracy and loss

# plot predictions 

# calculate TEST accuracy score 

# Calculate confusion matrices precision and recall 

### Phase 2: Freeze layer 4 - Fine tuning

# from the freezed cnn unfreeze the  ``` layer4 ```. 

# Train for 50 epochs 

# Plot curves

# visualize prediction

# Calculate test accuracy score 

# calculate confusion matrix precision and recall

# Question 2 What did you learn? What is the difference between transfer learning and fine tuning? 